# FORCE ESTIMATION PREPROCESSING

## Specify Paths and FLAGS

In [1]:
## TODO
# - unit convert to do but raw embedded into SI units (Si Machine) - DONE
# - jacobian preprocessing using interpolate jacobian (Si Machine) - DONE
# - split embeded data into force sensor ground truth (Si Machine ) - DONE
# - model prediction
# - procesing rosbag data through simons repo dvrk_force_estimation repo (Si Machine )
# - - rosbag reprocess (Si Machine)
# - - preprocess script(interpolations) (Si Machine)


# NOTES
#   make sure you put the json converted files in the jhu-davinci folders  

# file struture
# ---------------
# dataset_name 
# |__joints


## EMBEDDED PATHS
dataset_path = "/home/ndrakes1/data_collection_test/host/build/bin/capture_2-17-26/10khz/60_30hz_contact_pos_vel_pot_map/grip_fixture_aggressive_force/" # SET
dataset_name = "grip_fixture_aggressive_force"

fpga_v3_data_collection_path = "/home/ndrakes1/fpgav3-data-collection" # SET THIS ONCE
preproces_path = "/home/ndrakes1/force-estimation-fpga-data-preprocess"  # should be this directory SET THIS ONCE

true_preprocess_path = "~/force_estimation/preprocess/"


unit_convert_script_py = fpga_v3_data_collection_path + "/unit_convert/unit_convert.py"
pot_to_encoder_script_py = preproces_path + "/pot_to_encoder.py"
si_unit_xml_path = "/home/ndrakes1/catkin_ws/src/dvrk/dvrk_config_jhu/jhu-daVinci-Si/sawRobotIO1394-PSM3-698534.xml" # SET THIS ONCE 
si_unit_json_path = "/home/ndrakes1/catkin_ws/src/dvrk/dvrk_config_jhu/jhu-daVinci-Si/sawRobotIO1394-PSM1-292409.xml.json"

zynq_data_csv = "/home/ndrakes1/data_collection_test/host/build/bin/capture_2-17-26/10khz/60_30hz_contact_pos_vel_pot_map/grip_fixture_aggressive_force/grip_fixture_aggressive_force.csv"
zynq_data_pot_csv = dataset_path + "/" + dataset_name + "_potEncoder.csv"


unit_convert_path_default = dataset_path + "/" + dataset_name + "_unitConvert.csv"
unit_convert_path_pot = zynq_data_pot_csv.replace(".csv", "_unitConvert.csv")


ORIGINAL_FREQ = 10000

INTERPOLATE = True


FILTER = True
FILTER_FREQ = 30
FILTER_VELOCITY = True
FILTER_POSITION = True

DOWNSAMPLE = True
DOWNSAMPLE_FREQ = 60
DOWNSAMPLE_MOVING_AVERAGE = True

USE_POTS = True

if USE_POTS:
    unit_convert_path = unit_convert_path_pot
else:
    unit_convert_path = unit_convert_path_default

##################################################################################

# EMBEDDED

In [2]:
!. ~/catkin_ws/devel/setup.bash
from downsample import downsample_dataframe
import pandas as pd 
import os

## Unit Convert

In [3]:
if USE_POTS:
    !.venv/bin/python3 {pot_to_encoder_script_py} {zynq_data_csv} {zynq_data_pot_csv}
    !.venv/bin/python3 {unit_convert_script_py} -c {si_unit_json_path} -f {zynq_data_pot_csv}
else:
    !.venv/bin/python3 {unit_convert_script_py} -c {si_unit_json_path} -f {zynq_data_csv}

Traceback (most recent call last):
  File "/home/ndrakes1/force-estimation-fpga-data-preprocess/pot_to_encoder.py", line 6, in <module>
    def _fit_linear_map(x: pd.Series, y: pd.Series) -> tuple[float, float]:
TypeError: 'type' object is not subscriptable
Traceback (most recent call last):
  File "/home/ndrakes1/fpgav3-data-collection/unit_convert/unit_convert.py", line 115, in <module>
    main()
  File "/home/ndrakes1/fpgav3-data-collection/unit_convert/unit_convert.py", line 110, in main
    dataConverted = unitConvert(args.configuration, args.file_name)
  File "/home/ndrakes1/fpgav3-data-collection/unit_convert/unit_convert.py", line 15, in unitConvert
    robot_data = np.genfromtxt(fileName, delimiter=",", dtype=None, names=True,encoding='utf-8')
  File "/home/ndrakes1/force-estimation-fpga-data-preprocess/.venv/lib/python3.8/site-packages/numpy/lib/npyio.py", line 1977, in genfromtxt
    fid = np.lib._datasource.open(fname, 'rt', encoding=encoding)
  File "/home/ndrakes1/force-

## Extract Force Sensor Data

In [4]:
import filter
from interpolate_timestamps import interpolate_dataframe_to_sample_rate

!mkdir {dataset_path}/sensor
!.venv/bin/python3 sensor_processing.py {zynq_data_csv} {dataset_path}/sensor/sensor.csv

df = pd.read_csv( os.path.join(dataset_path,"sensor","sensor.csv") , header=None)

### FILTERING Force Sensor 

if (FILTER):

    fir_coeffs = filter.design_fir_filter(filter_type='kaiser', fs=ORIGINAL_FREQ, fC=FILTER_FREQ, order=30)
    df = filter.apply_filter_to_fs_df(df, fir_coeffs)

if (DOWNSAMPLE):
    df = downsample_dataframe(df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)

if (INTERPOLATE):
    freq = 0

    if (DOWNSAMPLE):
        freq = DOWNSAMPLE_FREQ
        print("INTERPOLATE DOWNSAMPLE")
    else:
        freq = ORIGINAL_FREQ
    
    df = interpolate_dataframe_to_sample_rate(df, freq)

df.to_csv(os.path.join(dataset_path,"sensor","sensor.csv"), index=False, header=False)



Saved SENSOR DATA CSV to /home/ndrakes1/data_collection_test/host/build/bin/capture_2-17-26/10khz/60_30hz_contact_pos_vel_pot_map/grip_fixture_aggressive_force//sensor/sensor.csv
INTERPOLATE DOWNSAMPLE
[0.00000000e+00 1.65853560e-02 3.31854271e-02 ... 1.12083185e+02
 1.12099785e+02 1.12116385e+02]


## Extract Joint Data

In [5]:
!mkdir {dataset_path}joints
!.venv/bin/python3 preprocessing.py {unit_convert_path} {dataset_path}joints/interpolated_all_joints.csv

Traceback (most recent call last):
  File "preprocessing.py", line 25, in <module>
    preprocess_csv(args.fileA, args.output_path)
  File "preprocessing.py", line 6, in preprocess_csv
    df = pd.read_csv(input_csv_path)
  File "/home/ndrakes1/force-estimation-fpga-data-preprocess/.venv/lib/python3.8/site-packages/pandas/io/parsers/readers.py", line 912, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/home/ndrakes1/force-estimation-fpga-data-preprocess/.venv/lib/python3.8/site-packages/pandas/io/parsers/readers.py", line 577, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/home/ndrakes1/force-estimation-fpga-data-preprocess/.venv/lib/python3.8/site-packages/pandas/io/parsers/readers.py", line 1407, in __init__
    self._engine = self._make_engine(f, self.engine)
  File "/home/ndrakes1/force-estimation-fpga-data-preprocess/.venv/lib/python3.8/site-packages/pandas/io/parsers/readers.py", line 1661, in _make_engine
    self.handles = get_handle(


## Filter Joint Data

### Preprocessed Training File Paths

In [6]:
import os
import pandas as pd

data_file = os.path.join(dataset_path,"joints" ,"interpolated_all_joints.csv")
df = pd.read_csv(data_file, header=None)


FileNotFoundError: [Errno 2] No such file or directory: '/home/ndrakes1/data_collection_test/host/build/bin/capture_2-17-26/10khz/60_30hz_contact_pos_vel_pot_map/grip_fixture_aggressive_force/joints/interpolated_all_joints.csv'

### FIR Filtering

In [ ]:
import importlib
import filter  # regular import first
importlib.reload(filter)


if (FILTER == True):
    # Design filter
    fir_coeffs = filter.design_fir_filter(filter_type='kaiser', fs=ORIGINAL_FREQ, fC=FILTER_FREQ, order=30)

    # Apply to data
    df = filter.apply_filter_to_torque_feedback_df(df, fir_coeffs, filter_velocity=FILTER_VELOCITY, filter_position=FILTER_POSITION)

### Downsample


In [ ]:


if (DOWNSAMPLE == True):
    df = downsample_dataframe(df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)


### Interpolate

In [ ]:
from interpolate_timestamps import interpolate_dataframe_to_sample_rate

if (INTERPOLATE == True):

    freq = 0

    if (DOWNSAMPLE):
        freq = DOWNSAMPLE_FREQ
    else:
        freq = ORIGINAL_FREQ
    
    df = interpolate_dataframe_to_sample_rate(df, freq)


[0.00000000e+00 1.65853560e-02 3.31854271e-02 ... 1.12083185e+02
 1.12099785e+02 1.12116385e+02]


### SAVE TO CSV

In [ ]:
df.to_csv(data_file, index=False, header=False)

# Extract Jacobian

In [ ]:
!mkdir {dataset_path}jacobian

data_file = os.path.join(dataset_path,"joints" ,"interpolated_all_joints.csv")

!.venv/bin/python3 interpolate_jacobian2.py {data_file} {dataset_path}jacobian/interpolated_all_jacobian.csv dvpsm.rob


mkdir: cannot create directory ‘/home/ndrakes1/data_collection_test/host/build/bin/capture_2-17-26/10khz/60_30hz_contact_pos_vel_pot/grip_fixture_aggressive_force/jacobian’: File exists
FIRST JACOBIAN VALUE
[[ 2.08166817e-17 -2.77555756e-17  1.12544429e-02 -1.81061763e-17
  -1.33836987e-01 -9.09632150e-02]
 [ 1.38777878e-17  0.00000000e+00  7.45319041e-01 -8.29414662e-18
   5.84873502e-02 -9.35681894e-02]
 [ 2.77555756e-17 -4.16333634e-17 -6.66612980e-01  2.43945489e-18
   6.31332967e-02 -1.06151389e-01]
 [-1.66533454e-16 -9.99857512e-01  0.00000000e+00  1.12544429e-02
  -5.40744742e-01  8.41143047e-01]
 [-1.00000000e+00 -5.55111512e-17  0.00000000e+00  7.45319041e-01
  -5.56230411e-01 -3.65191602e-01]
 [ 1.11022302e-16 -1.68806183e-02  0.00000000e+00 -6.66612980e-01
  -6.31033163e-01 -3.98890296e-01]]
Flattened Jacobians (column-major) written to /home/ndrakes1/data_collection_test/host/build/bin/capture_2-17-26/10khz/60_30hz_contact_pos_vel_pot/grip_fixture_aggressive_force/jacobian/

## Interpolate

In [ ]:


if (INTERPOLATE == True):

    freq = 0

    if (DOWNSAMPLE):
        freq = DOWNSAMPLE_FREQ
    else:
        freq = ORIGINAL_FREQ
    
    
    jacobian_path = os.path.join(dataset_path,"jacobian" ,"interpolated_all_jacobian.csv")
    df = pd.read_csv( jacobian_path, header=None)
    df = interpolate_dataframe_to_sample_rate(df, freq)
    df.to_csv(jacobian_path, index=False, header=False)


[0.00000000e+00 1.66691028e-02 3.33382056e-02 ... 1.12083047e+02
 1.12099716e+02 1.12116385e+02]


In [ ]:
# REMOVING JACOBIAN FILTERING -> MATHEMATICALLY INCORRECT


# df = pd.read_csv(os.path.join(dataset_path,"jacobian", "interpolated_all_jacobian.csv"), header=None)

# if DOWNSAMPLE == True:
#     df = downsample_dataframe(df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)
#     df.to_csv(os.path.join(dataset_path,"jacobian", "interpolated_all_jacobian.csv"), index=False, header=False)

# NOW DO INFERENCE ON THE JOINT DATA ON THE DVRK-12-MACHINE

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

csv_path = "/home/ndrakes1/data_collection_test/host/build/bin/capture_2-17-26/10khz/60_30hz_contact_pos_vel_pot_map/grip_fixture_aggressive_force/grip_fixture_aggressive_force_potEncoder.csv"
df = pd.read_csv(csv_path)

t = pd.to_numeric(df["TIMESTAMP"], errors="coerce")
t = t - t.iloc[0]

fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)
pairs = [("POT_3", "ENCODER_POS_1", "ORIGINAL_ENCODER_POS_1"),
         ("POT_4", "ENCODER_POS_2", "ORIGINAL_ENCODER_POS_2"),
         ("POT_5", "ENCODER_POS_3", "ORIGINAL_ENCODER_POS_3")]

for i, (pot, enc, enc_raw) in enumerate(pairs, start=1):
    ax = axes[i - 1]
    ax.plot(t, df[pot], label=pot, linewidth=0.9)
    ax.plot(t, df[enc], label=f"{enc} (mapped)", linewidth=0.9)
    ax.plot(t, df[enc_raw], label=f"{enc_raw} (raw)", linewidth=0.9, alpha=0.7)
    ax.set_title(f"Joint {i}")
    ax.grid(alpha=0.25)
    ax.legend(loc="upper right", fontsize=8)

axes[-1].set_xlabel("Time from start")
fig.tight_layout()
plt.show()

